In [2]:
!pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)


In [ ]:
import subprocess
import json
import requests
import ipaddress

def get_current_authorized_networks():
    result = subprocess.run([
        "gcloud", "sql", "instances", "describe", "bewantedsql",
        "--project=mystical-height-152415",
        "--format=json(settings.ipConfiguration.authorizedNetworks)"
    ], capture_output=True, text=True)
    
    try:
        data = json.loads(result.stdout)
        return data.get('settings', {}).get('ipConfiguration', {}).get('authorizedNetworks', [])
    except json.JSONDecodeError:
        print("Error decoding JSON from gcloud command")
        return []

def get_microsoft_ip_ranges():
    url = 'https://raw.githubusercontent.com/femueller/cloud-ip-ranges/master/microsoft-azure-ip-ranges.json'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        relevant_services = [
            'PowerBI', 'AzureCloud', 'AzureDataLake', 'PowerPlatformInfra',
            'PowerPlatformPlex', 'DataFactory', 'AzureFrontDoor',
            'SqlManagement', 'SQL'
        ]
        relevant_ips = []
        for item in data['values']:
            if any(service.lower() in item['name'].lower() for service in relevant_services):
                for ip in item['properties']['addressPrefixes']:
                    if is_valid_ipv4_cidr(ip):
                        relevant_ips.append((ip, item['name']))
        return relevant_ips
    except requests.RequestException as e:
        print(f"Error fetching IP ranges: {str(e)}")
        return []

def is_valid_ipv4_cidr(ip):
    try:
        network = ipaddress.IPv4Network(ip)
        return True
    except:
        return False

def update_authorized_networks():
    current_networks = get_current_authorized_networks()
    microsoft_ips = get_microsoft_ip_ranges()

    print("IPs actualmente autorizadas:")
    for net in current_networks:
        print(f"- {net['value']} ({net.get('name', 'Sin nombre')})")

    print("\nIPs de Microsoft relevantes:")
    current_ip_set = set(net['value'] for net in current_networks)
    new_ips = []
    for ip, service in microsoft_ips:
        if ip not in current_ip_set:
            new_ips.append((ip, service))
        print(f"- {ip} ({service})")

    if new_ips:
        print("\nNuevas IPs de Microsoft que se añadirán:")
        for ip, service in new_ips:
            print(f"- {ip} ({service})")
    else:
        print("\nNo hay nuevas IPs de Microsoft para añadir.")

    updated_networks = current_networks + [{'kind': 'sql#aclEntry', 'value': ip, 'name': service[:50]} for ip, service in new_ips]
    authorized_networks = ','.join([f"{net['value']}" for net in updated_networks])

    print("\nComando que se ejecutaría:")
    print(f"gcloud sql instances patch bewantedsql --authorized-networks='{authorized_networks}' --project=mystical-height-152415 --quiet")

    update_command = [
        "gcloud", "sql", "instances", "patch", "bewantedsql",
        f"--authorized-networks={authorized_networks}",
        "--project=mystical-height-152415",
        "--quiet"
    ]
    result = subprocess.run(update_command, capture_output=True, text=True)
    if result.returncode == 0:
        print("Authorized networks updated successfully")
    else:
        print(f"Error updating authorized networks: {result.stderr}")

    return f'IP range update process completed. {len(updated_networks)} total IP ranges in the list.'

# Run the function
result = update_authorized_networks()
print(result)
